In [1]:
import pandas as pd
import numpy as np

In [2]:
from surprise import Dataset
from surprise import Reader

In [3]:
from UBCFBase import UBCFBase
from FS_Weight import FS_Weight
from surprise import KNNBasic

In [4]:
reader = Reader(line_format='user item rating', sep=':')

In [5]:
data = Dataset.load_from_file('new_usr_ratings.txt', reader=reader)

In [6]:
data.split(n_folds=3)

In [7]:
trainsets = []
for trainset, testset in data.folds():
    trainsets.append(trainset)

In [8]:
first_trainset = trainsets[0]

In [9]:
first_trainset.n_users

970

In [10]:
def f(df):
    df['hot_w'] = 1 / np.log(math.e + df['hot'] - 1)
    temp = df.groupby('comment_type')['hot_w'].sum()
    
    if 1 in temp.index:
        return temp[1] / len(temp)
    else:
        return 0

In [12]:
algo1 = KNNBasic()
algo2 = FS_Weight(beta=1)

In [13]:
algo1.train(first_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [14]:
algo2.train(first_trainset)

C:\Users\rico\Miniconda2\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the behavior similarity matrix...
Done computing behavior similarity matrix


In [15]:
algo1.sim

array([[ 1.        ,  0.08003766,  0.14991182, ...,  0.        ,
         0.2       ,  0.        ],
       [ 0.08003766,  1.        ,  0.12006319, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.14991182,  0.12006319,  1.        , ...,  0.        ,
         0.5       ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.2       ,  1.        ,  0.5       , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [16]:
algo2.fusion_sim

array([[ 1.        ,  0.08003766,  0.14991182, ...,  0.        ,
         0.2       ,  0.        ],
       [ 0.08003766,  1.        ,  0.12006319, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.14991182,  0.12006319,  1.        , ...,  0.        ,
         0.5       ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.2       ,  1.        ,  0.5       , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [3]:
np.arange?

In [7]:
np.arange(11) / 10.0

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])

In [9]:
np.arange(0, 1.1, 0.1)

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])